# Naive Behler's Model

* Author: Xin Chem
* Email: Bismarrck@me.com

This jupyter notebook will give a naive demo of Behler's neural network using optimized structures of $\mathrm{B}_{20}$ clusters. The inner relation between Behler's model and Alexandrova's DNN will also be discussed.

### References:

1. Behler, J. (2011). Phys Chem Chem Phys 13: 17930-17955.
2. Behler, J. (2015). International Journal of Quantum Chemistry, 115(16): 1032-1050.
3. Zhai, H. and A. N. Alexandrova (2016). Journal of Chemical Theory and Computation: acs.jctc.6b00994-00914.


## 1. Symmetry Functions

Instead of using cartesian coordinates or interatomic distances directly, Behler uses some symmetry functions to generated local fingerprints for atoms. The symmetry functions were first proposed in [2007](http://journals.aps.org/prl/abstract/10.1103/PhysRevLett.98.146401). In [2011](http://dx.doi.org/10.1063/1.3553717), he discussed five different types of symmetry functions. These functions are labeled as $G_{i}^{k}$ where k is 1 to 5.

### 1.1 Equations

The first three symmetry functions are radial functions, describing the radial environment of atom $i$:

$$G_{i}^{1} = \sum_{j}{f_{c}{(R_{ij})}}$$
$$G_{i}^{2} = \sum_{j}{e^{ -\eta(R_{ij} - R_s)^2 } \cdot f_{c}{(R_{ij})}}$$
$$G_{i}^{3} = \sum_{j}{cos(\kappa R_{ij}) \cdot f_{c}{(R_{ij})}}$$

The two angular functions, $G^4_i$ and $G^5_i$, are summations of cosine functions of the angles $\theta_{ijk} = acos(\frac{R_{ij}R_{ik}}{|R_{ij}| \cdot |R_{ik}|})$ centered at atom $i$. They are defined as:

$$G_{i}^{4} = 2^{1 - \zeta}{ \sum_{j, j \ne i} {\sum_{k, k \ne j, k \ne i}} {
     [1 + \lambda cos\theta_{ijk})^\zeta \cdot e^{-\eta(R_{ij}^2 + R_{jk}^2 + R_{ik}^2) 
     }\cdot f_{c}(R_{ij}) \cdot f_{c}(R_{jk}) \cdot f_{c}(R_{ik})]
} }$$

$$G_{i}^{5} = 2^{1 - \zeta}{ \sum_{j, j \ne i} {\sum_{k, k \ne j, k \ne i}} {
     [1 + \lambda cos\theta_{ijk})^\zeta \cdot e^{-\eta(R_{ij}^2 + R_{ik}^2) 
     }\cdot f_{c}(R_{ij}) \cdot f_{c}(R_{ik})]
} }$$

In these functions, $f_{c}(R)$ is a cutoff function. Given a cutoff value $R_{c}$, the output of $f_c(R_{ij})$ can be calculated with the following equation:

$$f_{c}(R_{ij}) = \left\{\begin{array}{c}
0.5 \cdot \left[ cos(\pi\frac{R_{ij}}{R_{c}}) + 1 \right] \qquad R_{ij} \le R_{c}\\ 
0 \qquad R_{ij} > R_{c}
\end{array}\right.$$

### 1.2 Amp

In Behler's implementation and later in [Amp](http://dx.doi.org/10.1016/j.cpc.2016.05.010), the radial function $G^2_i$ and the angular function $G^4_i$ are used. 

There are total 5 different empirical hyper-parameters used in these two functions:

1. $\eta$ for $G^2_i$
2. $\eta$, $\lambda$ and $\zeta$ for $G^4_i$.
3. $R_{c}$ as the cutoff radius for $f_{c}(R_{ij})$.

The default values of these hyper-paramters in Amp are:

* Radial $\eta$
     * ``[0.05, 4., 20., 80.]``
* Angular $\eta$
     * ``[0.005]``
* Angular $\lambda$
     * ``[+1., -1.]``
* Angular $\zeta$
     * ``[1., 4.]``
* The default cutoff radius in Amp is 6.5 angstroms.

**Note**: 
* $\eta$ for $G^2_i$ and $G^4_i$ are totally different!
* These hyper-parameters are atomic-species related. So for different kinds of atoms combinations, we (shall) choose different parameters.

Typically we use more than 1 radial and 1 angular symmetry function to describe atoms. In the following sections we will discuss the dimensions of the fingerprints.

#### 1.2.1 $\mathrm{B}_{20}$

For $B_{20}$ or such kinds of monoatomic clusters, the dimension of the fingerprints for each atom is fixed to 8:

* 4 radial fingerprints, as there are 4 different $\eta$.
* 4 angular fingerprints, as there are 1x2x2 different combinations of ($\eta$, $\lambda$, $\zeta$).

There is only one possible atomic symbols combination for both $G^2_i$ and $G^4_{i}$:

* B-B for $G^2_i$
* B-B-B for $G^4_i$

#### 1.2.2 $\mathrm{C}_{4}\mathrm{H}_{8}$

For molecules containing two kinds of atoms, the dimension of the fingerprints for each kind of atom can be computed with the following procedures:

1. Radial:
     * C-C: 4
     * C-H: 4
     * H-H: 4
2. Angular:
     * C-C-C: 4
     * C-C-H: 4
     * C-H-H: 4
     * H-H-H: 4
3. The total dimensions of the fingerprints for $\mathrm{C}$ and $\mathrm{H}$:
     * C: **C-C** + **C-H** + **C-C-C** + **C-C-H** + **C-H-H** = 20
     * H: **H-H** + **C-H** + **H-H-H** + **C-C-H** + **C-H-H** = 20

#### 1.2.3 $\mathrm{C}_9\mathrm{H}_7\mathrm{N}$

Using the same method, we can get the dimensions of the fingerprints for $\mathrm{C}_9\mathrm{H}_7\mathrm{N}$:

* C
     * C-C: 4
     * C-H: 4
     * C-N: 4
     * C-C-C: 4
     * C-C-H: 4
     * C-H-H: 4
     * C-C-N: 4
     * C-H-N: 4
     * Total: 32
* H
     * H-H: 4
     * C-H: 4
     * N-H: 4
     * H-H-H: 4
     * C-H-H: 4
     * C-C-H: 4
     * C-H-N: 4
     * H-H-N: 4
     * Total: 32
* N
     * C-N: 4
     * H-N: 4
     * C-C-N: 4
     * C-H-N: 4
     * H-H-N: 4
     * Total: 20

## 2. The Neural Network

After calculating the fingerprints, Behler uses the standard feed-forward neural network to estimate energies. The overall structure of his network is demonstrated in the following [figure](http://pubs.rsc.org/en/Content/ArticleLanding/2011/CP/c1cp21668f):

<img src="./images/behler.png" width=500px>

The fingerprints calculated by the symmetry functions are **local** features, so the neural networks are **atom-wise**, but not **molecule-wise**. The estimated total energy, $E^{total}$, is the sum of energy of each atom. The cost function, excluding the force term, is:

$$\mathrm{RMSE} = \sqrt{
     \frac{1}{2}\sum_{i}^{N^{traj}}{\left[\left(\sum_{j}^{N^{atom}}{E_{i,j}}\right) - \hat{E_{i}}\right]^2}}
$$

where:

* $N^{traj}$ is the total number of trajectories.
* $N^{atom}$ is the number of atoms in each configuration.
* $\hat{E^{i}}$ is the DFT energy of configuration $i$.
* $E_{i,j}$ is the so called, "atomic energy", the estimted energy of atom $j$ of configration $i$.

**Note: the force term is ignored in this demo since it is extremely complicated!**

## 3. Case Study: $\mathrm{B}_{20}$

Let's consider the simplest case, a monoatomic cluster, $\mathrm{B}_{20}$. There is only one atomic neural network to train for this cluster as it just contains Boron atoms.

As described in **1.2.1**, each Boron atom is described by 8 fingerprints, including 4 radial features and 4 angular features. Suppose we use 2 hidden layers and each hidden layer has 100 units, the total number of parameters in this feed-forward network can be easily computed:

* Input-Hidden: 
     * 8x100 + 100 = 900
* Hidden-Hidden:
     * 100x100 + 100 = 10100
* Hidden-Output: 
     * 100x1 = 100
* Total: 11100

The Python codes to run **Amp** on this case:

```
from amp import Amp
from amp.descriptor.gaussian import Gaussian
from amp.regression.neuralnetwork import NeuralNetwork

f = Trajectory("./B20.traj")
images = [image for image in f]
f.close()

print(images)

o = Amp(descriptor=Gaussian(),
     regression=NeuralNetwork(hiddenlayers=(10, 10)),
     label="Behler",
     fortran=False)

o.train(images)
```

## 4. Naive Implementation of Symmetry Functions

There are tens of thousands of lines in **Amp**, so I implemented a naive version for $\mathrm{B}_{20}$ or such monoatomic clusters.

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers.python.layers import conv2d, flatten
from sklearn.metrics import pairwise_distances
from itertools import product
from pymatgen.io.xyz import XYZ
from os.path import join

This helper function is used to detect whether $i$ and $j$ are neighbours.

In [2]:
def get_neighbour_list(r, cutoff):
  """
  A naive implementation of calculating neighbour list for non-periodic
  molecules.

  Args:
    r: a `[N, N]` array as the pairwise interatomic distances.
    cutoff: a float as the cutoff radius.

  Returns:
    neighbours: a `[N, N]` boolean array as the neighbour results.

  """
  neighbours = r < cutoff
  np.fill_diagonal(neighbours, False)
  return neighbours

An updated cutoff function. The origin cutoff function contains a ``if-else``: 
$$f_{c}(R_{ij}) = \left\{\begin{array}{c}
0.5 \cdot \left[ cos(\pi\frac{R_{ij}}{R_{c}}) + 1 \right] \qquad R_{ij} \le R_{c}\\ 
0 \qquad R_{ij} > R_{c}
\end{array}\right.$$

But we can use `min()` to vectorize this function:
$$f_{c}(R_{ij}) = 0.5 \cdot \left[ cos \left( \mathrm{min}( \frac{R_{ij}}{R_{c}}, 1) \cdot \pi \right) + 1\right]$$

In [3]:
def cutoff_fxn(r, rc):
  """
  The vectorized cutoff function.

  Args:
    r: a `float` or an array as the interatomic distances.
    rc: a `float` as the cutoff radius.

  Returns:
    fr: the damped `r`.

  """
  return (np.cos(np.minimum(r / rc, 1.0) * np.pi) + 1.0) * 0.5

The radial and angular fingerprints can be calculated with the following functions:

$$G_{i}^{2} = \sum_{j}{e^{ -\eta(R_{ij} - R_s)^2 } \cdot f_{c}{(R_{ij})}}$$

$$G_{i}^{4} = 2^{1 - \zeta}{ \sum_{j, j \ne i} {\sum_{k, k \ne j, k \ne i}} {
     [1 + \lambda cos\theta_{ijk})^\zeta \cdot e^{-\eta(R_{ij}^2 + R_{jk}^2 + R_{ik}^2) 
     }\cdot f_{c}(R_{ij}) \cdot f_{c}(R_{jk}) \cdot f_{c}(R_{ik})]
} }$$

**Note**: 

the $R_{s}$ term in $G^2-i$ represents the closest position of atom $j$ to the center atom $i$. For non-periodic molecules, $R_{s}$ is equal to $R_{j}$.

In [5]:
def get_radial_fingerprints(coords, r, rc, etas):
  """
  Return the fingerprints from the radial gaussian functions.

  Args:
    coords: a `[N, 3]` array as the cartesian coordinates.
    r: a `[N, N]` array as the pairwise distance matrix.
    rc: a float as the cutoff radius.
    etas: a `List[float]` as the `eta` in the radial functions.

  Returns:
    x: a `[N, M]` array as the radial fingerprints.

  """

  params = np.array(etas)
  ndim = len(params)
  natoms = len(coords)
  x = np.zeros((natoms, ndim))
  nl = get_neighbour_list(r, rc)
  rc2 = rc ** 2
  fr = cutoff_fxn(r, rc)

  for l, eta in enumerate(etas):
    for i in range(natoms):
      v = 0.0
      ri = coords[i]
      for j in range(natoms):
        if not nl[i, j]:
          continue
        rs = coords[j]
        ris = np.sum(np.square(ri - rs))
        v += np.exp(-eta * ris / rc2) * fr[i, j]
      x[i, l] = v
  return x


def get_augular_fingerprints_naive(coords, r, rc, etas, gammas, zetas):
  """
  Return the fingerprints from the augular functions.

  Args:
    coords: a `[N, 3]` array as the cartesian coordinates.
    r: a `[N, N]` matrix as the pairwise distances.
    rc: a float as the cutoff radius.
    etas: a `List[float]` as the `eta` in the radial functions.
    gammas: a `List[float]` as the `lambda` in the angular functions.
    zetas: a `List[float]` as the `zeta` in the angular functions.

  Returns:
    x: a `[N, M]` array as the augular fingerprints.

  """
  natoms = len(r)
  params = np.array(list(product(etas, gammas, zetas)))
  ndim = len(params)
  rr = r + np.eye(natoms) * rc
  r2 = rr ** 2
  rc2 = rc ** 2
  fr = cutoff_fxn(rr, rc)
  x = np.zeros((natoms, ndim))

  for l, (eta, gamma, zeta) in enumerate(params):
    for i in range(natoms):
      for j in range(natoms):
        if j == i:
          continue
        for k in range(natoms):
          if k == i or k == j:
            continue
          rij = coords[j] - coords[i]
          rik = coords[k] - coords[i]
          theta = np.dot(rij, rik) / (r[i, j] * r[i, k])
          v = (1 + gamma * theta)**zeta
          v *= np.exp(-eta * (r2[i, j] + r2[i, k] + r2[j, k]) / rc2)
          v *= fr[i, j] * fr[j, k] * fr[i, k]
          x[i, l] += v
    x[:, l] *= 2.0 **(1 - zeta)

  return x / 2.0

Now let's test the codes above. The `v` below is calculated by **Amp** with $R_{c}=6.5$ and all default hyper-parameters:

```
v = [
    7.4962211769419, 3.908699186062, 0.9606682539908, 0.02123383466,
    25.00382305735, 15.2551697332, 6.6248599213, 0.6102237982
]
```

In [6]:
def get_behler_fingerprints(coords, rc, radial_etas=None, angular_etas=None,
                            gammas=None, zetas=None):
  """
  Return the Behler's fingerprints.

  Args:
    coords: a `[N, 3]` array as the atomic coordinates.
    rc: a float as the cutoff radius.
    radial_etas: a `List[float]` as the `eta` in the radial functions.
    angular_etas: a `List[float]` as the `eta` in the angular functions.
    gammas: a `List[float]` as the `lambda` in the angular functions.
    zetas: a `List[float]` as the `zeta` in the angular functions.

  Returns:
    fingerprints: a `[N, M]` array as the fingerprints of the given molecule.

  """

  # These are default parameters defined by Behler. I just copied these from the
  # Amp source file `gaussian.py`.
  if radial_etas is None:
    radial_etas = [0.05, 4., 20., 80.]
  if angular_etas is None:
    angular_etas = [0.005]
  if gammas is None:
    gammas = [+1., -1.]
  if zetas is None:
    zetas = [1., 4.]

  r = pairwise_distances(coords)
  radials = get_radial_fingerprints(
    coords,
    r,
    rc,
    radial_etas
  )
  augulars = get_augular_fingerprints_naive(
    coords,
    r,
    rc,
    angular_etas,
    gammas, zetas
  )
  return np.hstack((radials, augulars))

In [8]:
xyzfile = join("..", "datasets", "B20pbe_opted.xyz")
cart_coords = XYZ.from_file(xyzfile).molecule.cart_coords
rc = 6.5
fingers = get_behler_fingerprints(cart_coords, rc)
v = [
  7.4962211769419, 3.908699186062, 0.9606682539908, 0.02123383466,
  25.00382305735, 15.2551697332, 6.6248599213, 0.6102237982
]
diff = np.linalg.norm(fingers[0] - v)
if diff < 1e-6:
  print("The test is passed!")
else:
  print("The test is failed!")

The test is passed!


## 5. Behler's is Alexandrova's 

**For mono-atomic clusters, the Behler's network can be expressed with Alexandrova's convolutional structure exactly.**

I will show this interesting relation step by step.

### 5.1 Behler's on $\mathrm{B}_{20}$

Suppose there are total N different structuress of $\mathrm{B}_{20}$ clusters. As described above, each $\mathrm{B}_{20}$ cluster has 20 atoms and each Boron atom has 8 fingerprints. If we store all features in a 3D array, the shape of the dataset array should be:

$$[N, 20, 8]$$

This is a monoatomic cluster, so there is only one atomic neural network to train in Behler's model. Suppose we use two hidden layers and each hidden layer has 100 units, the feed-forward atomic neural network can be expressed as the following:

<img src="./images/BehlerB20.png" width=600px>

There are three weights matrices in this feed-forward network:

* $W_{ih}$
     * Input->Hidden
     * [8, 100]
* $W_{hh}$
     * Hidden->Hidden
     * [100, 100]
* $W_{ho}$
     * Hidden->Output
     * [100, 1]

These weights matrices are shared by all atomic neural networks.

The total energy of configuration $i$ can be calculated with the equation below:

$$
E^{pred}_{i} = \sum_{j}^{20}{E^{i}_{j}} = \sum_{j}^{20}{\mathrm{NN}(x_{j}^i)} 
= \sum{ \left( \mathrm{NN}( X^{i} ) \right)}
$$

$$
X^{i} = \left[ \begin{array}{cccccc} 
x_{1}^i & x_{2}^i & x_{3}^i & x_{4}^i & \dots & x_{20}^i
\end{array}\right]^T
$$

### 5.2 The Math Inside

Suppose $x^i_{j}$ represents the fingerprints of atom $j$ in configuration $i$. The shape of this vector should be:

$$[1, 8]$$

Then we can get the values of the first hidden layer:

$$z^i_j = x^i_j W_{ih} + b_{ih}$$

The shape of $z^i_j$ should be **[1, 100]**.

For configuration $i$, we can write the above equation to its matrix form:

$$
z^i = 
\left[
\begin{array}{c}
x^i_1 W_{ih} + b_{ih}\\
x^i_2 W_{ih} + b_{ih}\\
x^i_3 W_{ih} + b_{ih}\\
\vdots \\
x^i_{20} W_{ih} + b_{ih}\\
\end{array}
\right]
$$

The $z^i$ above is a **matrix** of shape [20, 100]. The **first column** of $z^{i}$ can be expanded to:

$$ 
z^i = \left[\begin{array}{cc}
\sum_{k=1}^{8}{\left(x^i_{1,k} \cdot W^{ih}_{k,1}\right)} + b^{ih}_{1}, & \dots \\
\sum_{k=1}^{8}{\left(x^i_{2,k} \cdot W^{ih}_{k,1}\right)} + b^{ih}_{1}, & \dots \\
\sum_{k=1}^{8}{\left(x^i_{3,k} \cdot W^{ih}_{k,1}\right)} + b^{ih}_{1}, & \dots \\
\vdots & \\
\sum_{k=1}^{8}{\left(x^i_{20,k} \cdot W^{ih}_{k,1}\right)} + b^{ih}_{1}, & \dots \\
\end{array}\right]
$$

**The first column of $z^{i}$ is exactly the result of applying a 1x1 convolution kernel on $x^{i}$!**

<img src="./images/mathflow.png" width=700px>

The Input-to-Hidden weights and biases can be constructed by just stacking 100 such 1x1 convolution kernels.

The following Python codes just demonstrate how to conveniently build up a Behler's network using convolution layers. For more information please see `behler.py`.

In [16]:
def get_number_of_trainable_parameters(verbose=False):
  """
  Return the number of trainable parameters in current graph.

  Args:
    verbose: a bool. If True, the number of parameters for each variable will
    also be printed.

  Returns:
    ntotal: the total number of trainable parameters.

  """
  print("")
  print("Compute the total number of trainable parameters ...")
  print("")
  ntotal = 0
  for var in tf.trainable_variables():
    nvar = np.prod(var.get_shape().as_list(), dtype=int)
    if verbose:
      print("{:25s}  {:d}".format(var.name, nvar))
    ntotal += nvar
  print("")
  print("Total number of parameters: %d" % ntotal)

  
def inference(conv, activation=tf.nn.tanh, hidden_sizes=(100, 100)):
  """
  Infer the Behler's model for a monoatomic cluster.

  Args:
    conv: a `[-1, 1, N, M]` Tensor as the input. N is the number of atoms in
      the monoatomic cluster and M is the number of features.
    activation: the activation function. Defaults to `tf.nn.tanh`.
    hidden_sizes: List[int], the number of units of each hidden layer.

  Returns:
    total_energy: the output Tensor of shape `[-1, 1]` as the estimated total
      energies.
    atomic_energies: a Tensor of `[-1, N]`, the estimated energy for each atom.

  """
  assert len(hidden_sizes) >= 1

  kernel_size = 1
  stride = 1
  padding = 'SAME'

  for i, units in enumerate(hidden_sizes):
    conv = conv2d(
      conv,
      units,
      kernel_size,
      activation_fn=activation,
      stride=stride,
      padding=padding,
      scope="Hidden{:d}".format(i + 1),
    )

  atomic_energies = conv2d(
    conv,
    1,
    kernel_size,
    activation_fn=None,
    biases_initializer=None,
    stride=stride,
    padding=padding,
    scope="AtomEnergy"
  )

  flat = flatten(atomic_energies)
  total_energy = tf.reduce_sum(flat, axis=1, keep_dims=False)
  return total_energy, atomic_energies

Now let's check the total number of trainable parameters in this network:

In [17]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 1, 20, 8], name="input")
total_energy, _ = inference(x)
get_number_of_trainable_parameters(verbose=True)


Compute the total number of trainable parameters ...

Hidden1/weights:0          800
Hidden1/biases:0           100
Hidden2/weights:0          10000
Hidden2/biases:0           100
AtomEnergy/weights:0       100

Total number of parameters: 11100
